# Solutions: Python OOP Practice — Exercises 1–10
These are reference implementations with basic self-tests. Use them to compare against your own solutions.

## 1) Encapsulation + Properties — `BankAccount`

In [ ]:
class BankAccount:
    def __init__(self, owner: str, starting_balance: float = 0.0):
        if starting_balance < 0:
            raise ValueError("starting_balance must be >= 0")
        self.owner = owner
        self.__balance = float(starting_balance)
        self._log: list[tuple[str, float, float]] = []
        if starting_balance:
            self._add_log("open", starting_balance)

    @property
    def balance(self) -> float:
        return self.__balance

    def _add_log(self, kind: str, amount: float) -> None:
        self._log.append((kind, float(amount), float(self.__balance)))

    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("amount must be > 0")
        self.__balance += amount
        self._add_log("deposit", amount)

    def withdraw(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("amount must be > 0")
        if amount > self.__balance:
            raise ValueError("insufficient funds")
        self.__balance -= amount
        self._add_log("withdraw", amount)

    def statement(self) -> list[tuple[str, float, float]]:
        return list(self._log)

# Quick tests
acc = BankAccount("Alice", 100)
acc.deposit(50)
try:
    acc.withdraw(1000)
    assert False
except ValueError:
    pass
acc.withdraw(30)
st = acc.statement()
assert st[-1][2] == 120 and acc.balance == 120
print("BankAccount: OK")

## 2) Inheritance + ABC — `Shape`, `Rectangle`, `Circle`

In [ ]:
from abc import ABC, abstractmethod
import math

class Shape(ABC):
    @abstractmethod
    def area(self) -> float: ...
    @abstractmethod
    def perimeter(self) -> float: ...

class Rectangle(Shape):
    def __init__(self, w: float, h: float):
        self.w, self.h = float(w), float(h)
    def area(self) -> float:
        return self.w * self.h
    def perimeter(self) -> float:
        return 2 * (self.w + self.h)

class Circle(Shape):
    def __init__(self, r: float):
        self.r = float(r)
    def area(self) -> float:
        return math.pi * self.r * self.r
    def perimeter(self) -> float:
        return 2 * math.pi * self.r

r = Rectangle(3,4); c = Circle(1)
assert r.area() == 12 and r.perimeter() == 14
assert math.isclose(c.area(), math.pi, rel_tol=1e-6)
assert math.isclose(c.perimeter(), 2*math.pi, rel_tol=1e-6)
print("Shapes: OK")

## 3) Polymorphism — Animals

In [ ]:
class Animal:
    def speak(self) -> str:
        raise NotImplementedError

class Dog(Animal):
    def speak(self) -> str:
        return "Woof!"

class Cat(Animal):
    def speak(self) -> str:
        return "Meow!"

def chorus(animals: list[Animal]) -> list[str]:
    return [a.speak() for a in animals]

pets = [Dog(), Cat(), Dog()]
assert chorus(pets) == ["Woof!","Meow!","Woof!"]
print("Polymorphism: OK")

## 4) Composition — ShoppingCart

In [ ]:
class Product:
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = float(price)

class CartItem:
    def __init__(self, product: Product, quantity: int):
        if quantity <= 0:
            raise ValueError("qty must be > 0")
        self.product = product
        self.quantity = int(quantity)
    @property
    def subtotal(self) -> float:
        return self.product.price * self.quantity

class ShoppingCart:
    def __init__(self, tax_rate: float = 0.2):
        self.tax_rate = float(tax_rate)
        self._items: dict[str, CartItem] = {}
    def add_item(self, product: Product, qty: int) -> None:
        if product.name in self._items:
            self._items[product.name].quantity += qty
        else:
            self._items[product.name] = CartItem(product, qty)
    def remove_item(self, name: str) -> None:
        self._items.pop(name, None)
    def subtotal(self) -> float:
        return sum(item.subtotal for item in self._items.values())
    def tax(self) -> float:
        return self.subtotal() * self.tax_rate
    def total(self) -> float:
        return self.subtotal() + self.tax()

p1 = Product("Book", 10.0); p2 = Product("Pen", 1.5)
cart = ShoppingCart(tax_rate=0.19)
cart.add_item(p1, 2); cart.add_item(p2, 3)
st = cart.subtotal(); tx = cart.tax(); gt = cart.total()
assert st == 10*2 + 1.5*3 and round(tx,2) == round(st*0.19,2) and round(gt,2) == round(st+tx,2)
print("ShoppingCart: OK")

## 5) Class & Static Methods — Employee

In [ ]:
class Employee:
    company = "TechCorp"
    def __init__(self, name: str, salary: int):
        self.name = name
        self.salary = int(salary)
    @classmethod
    def change_company(cls, new: str) -> None:
        cls.company = new
    @staticmethod
    def is_high_salary(salary: int) -> bool:
        return salary > 100_000

e = Employee("John", 90_000)
assert Employee.company == "TechCorp"
Employee.change_company("NewTech")
assert e.company == "NewTech" and not Employee.is_high_salary(e.salary)
print("Employee: OK")

## 6) Dunder Methods — Vector

In [ ]:
class Vector:
    def __init__(self, x: float, y: float):
        self.x, self.y = float(x), float(y)
    def __add__(self, other: 'Vector') -> 'Vector':
        if not isinstance(other, Vector):
            return NotImplemented
        return Vector(self.x + other.x, self.y + other.y)
    def __eq__(self, other: object) -> bool:
        return isinstance(other, Vector) and self.x == other.x and self.y == other.y
    def __repr__(self) -> str:
        return f"Vector({int(self.x) if self.x.is_integer() else self.x}, {int(self.y) if self.y.is_integer() else self.y})"

v1 = Vector(2,3); v2 = Vector(4,5)
assert (v1+v2) == Vector(6,8)
assert repr(v1) == "Vector(2, 3)"
print("Vector: OK")

## 7) Multiple Inheritance & MRO — Tracer

In [2]:
class A:
    def greet(self) -> str:
        return "Hello from A"
class B:
    def greet(self) -> str:
        return "Hello from B"
class Mixin:
    def __init__(self):
        self.trace = []
        super().__init__()

class C(Mixin, A, B):
    def __init__(self):
        super().__init__()
    def greet(self) -> str:
        self.trace.append("C.greet")
        # Call next in MRO (A.greet next)
        return super().greet()

c = C()
msg = c.greet()
assert msg in {"Hello from A","Hello from B"}
assert "C.greet" in c.trace
_ = C.mro()
print("MRO/Tracer: OK")

MRO/Tracer: OK


## 8) Dependency Injection + Mocking — ApiService

In [ ]:
class ApiService:
    def __init__(self, client):
        self._client = client
    def get_user_name(self, user_id: int) -> str:
        payload = self._client.get(f"/users/{user_id}")
        return payload["name"]

class MockClient:
    def __init__(self, data):
        self.data = data
        self.calls = []
    def get(self, url: str) -> dict:
        self.calls.append(url)
        return self.data[url]

mock = MockClient({"/users/1": {"id":1,"name":"Alice"}})
svc = ApiService(mock)
assert svc.get_user_name(1) == "Alice"
assert mock.calls == ["/users/1"]
print("ApiService: OK")

## 9) Dataclasses — Point (frozen)

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Point:
    x: int
    y: int
    def manhattan(self, other: 'Point') -> int:
        return abs(self.x - other.x) + abs(self.y - other.y)

p1 = Point(1,2); p2 = Point(4,6)
assert p1.manhattan(p2) == 7
try:
    p1.x = 99
    assert False
except Exception:
    pass
print("Point: OK")

## 10) Strategy Logger — ConsoleHandler & MemoryHandler

In [ ]:
class Handler:
    def emit(self, level: str, msg: str) -> None:
        raise NotImplementedError

class ConsoleHandler(Handler):
    def emit(self, level: str, msg: str) -> None:
        print(f"[{level}] {msg}")

class MemoryHandler(Handler):
    def __init__(self):
        self.store: list[tuple[str,str]] = []
    def emit(self, level: str, msg: str) -> None:
        self.store.append((level, msg))

class Logger:
    def __init__(self, level_filter: set[str], handlers: list[Handler]):
        self.level_filter = set(level_filter)
        self.handlers = list(handlers)
    def log(self, level: str, msg: str) -> None:
        if level not in self.level_filter:
            return
        for h in self.handlers:
            h.emit(level, msg)

mem = MemoryHandler()
logger = Logger(level_filter={"INFO","ERROR"}, handlers=[mem])
logger.log("DEBUG","x")
logger.log("INFO","hello")
assert mem.store == [("INFO","hello")]
print("Logger: OK")